In [1]:
%cd ..
from model.base import CompositeModel, FeatureEncodingPart, NodeRepresentationPart
from relbench.modeling.nn import HeteroTemporalEncoder

/home/lingze/embedding_fusion


/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import relbench
from relbench.base import Table, Database, Dataset, EntityTask
from relbench.datasets import get_dataset
from relbench.tasks import get_task
from relbench.base import BaseTask
from torch_geometric.seed import seed_everything
from relbench.modeling.utils import get_stype_proposal
from relbench.modeling.graph import make_pkey_fkey_graph
from relbench.modeling.graph import get_node_train_table_input
from torch_geometric.loader import NeighborLoader

import os
import math
import numpy as np
from tqdm import tqdm
import copy

import torch
from torch import Tensor
import torch_geometric
import torch_frame

from torch_frame.config.text_embedder import TextEmbedderConfig
from typing import List, Optional
from sentence_transformers import SentenceTransformer

from torch.nn import L1Loss, BCEWithLogitsLoss
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

seed_everything(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
dataset = get_dataset(name="rel-trial", download=True)
db = dataset.get_db()
task_a = get_task("rel-trial", "study-outcome", download = True)
task_b = get_task("rel-trial", "site-success", download = True)

Loading Database object from /home/lingze/.cache/relbench/rel-trial/db...
Done in 8.01 seconds.


In [4]:
col_to_stype_dict = get_stype_proposal(db)

class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device
                                       ] = None):
        self.model = SentenceTransformer(
            "sentence-transformers/average_word_embeddings_glove.6B.300d",
            device=device,
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return torch.from_numpy(self.model.encode(sentences))

text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device), batch_size=256
)


root_dir = "/home/lingze/embedding_fusion/data"
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,  # speficied column types
    text_embedder_cfg=text_embedder_cfg,  # our chosen text encoder
    cache_dir=os.path.join(
        root_dir, f"rel-trial_materialized_cache"
    ),  # store materialized graph for convenience
)

/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Multi-task joint train with shared feature encoder.


In [5]:
def generate_loader_dict(task: BaseTask) -> dict:
    loader_dict = {}
    for split, table in [
        ("train", task.get_table("train")),
        ("val",task.get_table("val")),
        ("test", task.get_table("test")),
    ]:
        table_input = get_node_train_table_input(
            table=table,
            task=task,
        )
        loader_dict[split] = NeighborLoader(
            data,
            num_neighbors=[
                128 for i in range(2)
            ],  # we sample subgraphs of depth 2, 128 neighbors per node.
            time_attr="time",
            input_nodes=table_input.nodes,
            input_time=table_input.time,
            transform=table_input.transform,
            batch_size=512,
            temporal_strategy="uniform",
            shuffle=split == "train",
            num_workers=0,
            persistent_workers=False,
        )
    return loader_dict

In [6]:
taska_loader_dict = generate_loader_dict(task_a)
taskb_loader_dict = generate_loader_dict(task_b)

In [8]:
channels = 128
temporal_encoder = HeteroTemporalEncoder(
    node_types=[
                node_type for node_type in data.node_types if "time" in data[node_type]
            ],
            channels=channels,
)

feat_encoder = FeatureEncodingPart(
    data=data,
    node_to_col_stats=col_stats_dict,
    channels=channels
)

task_a_node_encoder = NodeRepresentationPart(
    data=data,
    channels=channels,
    num_layers=1,
    normalization="layer_norm",
    dropout_prob=0.2
)

task_b_node_encoder = NodeRepresentationPart(
    data=data,
    channels=channels,
    num_layers=1,
    normalization="layer_norm",
    dropout_prob=0.4
)

task_a_model =  CompositeModel(
    data=data,
    channels=channels,
    out_channels=1,
    dropout=0.2,
    aggr="mean",
    norm="layer_norm",
    num_layer=2,
    feature_encoder=feat_encoder,
    node_encoder=task_a_node_encoder,
    temporal_encoder=temporal_encoder
)

task_b_model =  CompositeModel(
    data=data,
    channels=channels,
    out_channels=1,
    dropout=0.3,
    aggr="mean",
    norm="layer_norm",
    num_layer=2,
    feature_encoder=feat_encoder,
    node_encoder=task_b_node_encoder,
    temporal_encoder=temporal_encoder
)

In [9]:
@torch.no_grad()
def valid(loader: NeighborLoader, model: torch.nn.Module, task: BaseTask)-> np.ndarray:
    model.eval()
    pred_list = []
    pred_hat_list = []
    for batch in loader:
        batch = batch.to(device)
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
        pred_hat_list.append(batch[task.entity_table].y.detach().cpu())
    return torch.cat(pred_list, dim=0), torch.cat(pred_hat_list, dim=0)

@torch.no_grad()
def test(loader: NeighborLoader, model: torch.nn.Module, task: BaseTask)-> np.ndarray:
    model.eval()
    pred_list = []
    for batch in loader:
        batch = batch.to(device)
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
    return torch.cat(pred_list, dim=0)

In [27]:
params = set(list(task_a_model.parameters()) + list(task_b_model.parameters()))
optimizer = torch.optim.Adam(params, lr = 0.005)
epochs = 20
task_a_loss_fn = BCEWithLogitsLoss()
task_b_loss_fn = L1Loss()

task_a_model.to(device)
task_b_model.to(device)

batch_size = 512
higher_is_better = False
state_dict_a = state_dict_b = None
best_val_metric = float("-inf") if higher_is_better else float("inf")
best_epoch = None
early_stop = 20

task_a_model.reset_parameters()
task_b_model.reset_parameters()


for epoch in range(1, epochs + 1):
    task_a_model.train(), task_b_model.train()
    loss_a_accum = loss_b_accum = count_accum = loss = 0
    freshape = lambda x: x.view(-1) if x.size(1) == 1 else x
    cnt = 0
    for batch_a, batch_b in tqdm(zip(taska_loader_dict["train"], taskb_loader_dict["train"])):
        cnt += 1
        if cnt > early_stop:
            break
        
        batch_a, batch_b = batch_a.to(device), batch_b.to(device)
        pred_a = task_a_model(
            batch_a,
            task_a.entity_table,
        )
        pred_b = task_b_model(
            batch_b,
            task_b.entity_table,
        )
        pred_a = freshape(pred_a)
        pred_b = freshape(pred_b)
         
        optimizer.zero_grad()
        loss_a = task_a_loss_fn(
            pred_a.float(),
            batch_a[task_a.entity_table].y.float(),
        )
        loss_b = task_b_loss_fn(
            pred_b.float(),
            batch_b[task_b.entity_table].y.float(),
        )
        loss = loss_a + loss_b
            
        loss.backward()
        optimizer.step() 

        loss_a_accum += loss_a.detach().item()* batch_size
        loss_b_accum += loss_b.detach().item()* batch_size
        loss = loss.detach().item()* batch_size
        count_accum += batch_size
    
    train_loss = loss / count_accum
    train_loss_a = loss_a_accum / count_accum
    train_loss_b = loss_b_accum / count_accum
    
    val_a_logits, val_a_hat = valid(taska_loader_dict["val"], task_a_model, task_a)
    val_a_loss = task_a_loss_fn(
        val_a_logits.float(),
        val_a_hat.float()
    )

    val_a_logits = torch.sigmoid(val_a_logits).numpy()
    val_a_pred = (val_a_logits > 0.5).astype(int)
    val_a_pred_hat = task_a.get_table("val").df[task_a.target_col].to_numpy()
    val_a_metrics = {
        "auroc": roc_auc_score(val_a_pred_hat, val_a_logits),
        "accuracy": accuracy_score(val_a_pred_hat, val_a_pred),
        "precision": precision_score(val_a_pred_hat, val_a_pred),
        "recall": recall_score(val_a_pred_hat, val_a_pred),
        "f1score": f1_score(val_a_pred_hat, val_a_pred),
        "loss": val_a_loss.item()
    }
    
    val_b_logits, val_b_hat = valid(taskb_loader_dict["val"], task_b_model, task_b)
    val_b_loss = task_b_loss_fn(
        val_b_logits.float(),
        val_b_hat.float()
    )
    
    val_b_logits = val_b_logits.numpy()
    val_b_pred_hat = task_b.get_table("val").df[task_b.target_col].to_numpy()
    
    val_b_metrics = {
        "mae": mean_absolute_error(val_b_pred_hat, val_b_logits),
        "r2": r2_score(val_b_pred_hat, val_b_logits),
        "rmse": root_mean_squared_error(val_b_pred_hat, val_b_logits),
        "loss": val_b_loss.item()
    }
    
    eval_metric = val_a_loss + val_b_loss
    
    
    print("*"*30 + f"<Epoch: {epoch:02d}>" + "*"*30)
    print(f", Train loss: {train_loss}, Val metrics: {eval_metric}")
    print(f"Task A: Train loss: {train_loss_a}, Val metrics: {val_a_metrics}")
    print(f"Task B: Train loss: {train_loss_b}, Val metrics: {val_b_metrics}")
    
    
    if (higher_is_better and eval_metric > best_val_metric) or (
        not higher_is_better and eval_metric < best_val_metric
    ):
        best_val_metric = eval_metric
        best_epoch = epoch
        state_dict_a = copy.deepcopy(task_a_model.state_dict())
        state_dict_b = copy.deepcopy(task_b_model.state_dict())

0it [00:00, ?it/s]

20it [00:04,  4.01it/s]


******************************<Epoch: 01>******************************
, Train loss: 0.05026651620864868, Val metrics: 1.1287024021148682
Task A: Train loss: 0.5926268965005874, Val metrics: {'auroc': 0.6309490303298353, 'accuracy': 0.6072916666666667, 'precision': 0.6762452107279694, 'recall': 0.6292335115864528, 'f1score': 0.6518928901200369, 'loss': 0.6830201745033264}
Task B: Train loss: 0.5859857603907586, Val metrics: {'mae': 0.44568227610100564, 'r2': -0.13410892681006503, 'rmse': 0.5087180586627122, 'loss': 0.44568225741386414}


20it [00:05,  3.71it/s]


******************************<Epoch: 02>******************************
, Train loss: 0.042929285764694215, Val metrics: 1.1156361103057861
Task A: Train loss: 0.5540477871894837, Val metrics: {'auroc': 0.6301672184025126, 'accuracy': 0.6010416666666667, 'precision': 0.6444805194805194, 'recall': 0.7076648841354723, 'f1score': 0.6745964316057774, 'loss': 0.7034967541694641}
Task B: Train loss: 0.33491129279136655, Val metrics: {'mae': 0.41213944470760594, 'r2': -0.21680619809192425, 'rmse': 0.52693916247157, 'loss': 0.4121394157409668}


...

******************************<Epoch: 18>******************************
, Train loss: 0.02933640480041504, Val metrics: 1.3044493198394775
Task A: Train loss: 0.37473154366016387, Val metrics: {'auroc': 0.6247838848458043, 'accuracy': 0.6041666666666666, 'precision': 0.6579406631762653, 'recall': 0.6720142602495544, 'f1score': 0.6649029982363316, 'loss': 0.8964056372642517}
Task B: Train loss: 0.22928790375590324, Val metrics: {'mae': 0.4080436767650708, 'r2': -0.509689026452276, 'rmse': 0.5869397454280564, 'loss': 0.40804368257522583}


20it [00:05,  3.68it/s]


******************************<Epoch: 19>******************************
, Train loss: 0.030465489625930785, Val metrics: 1.4638752937316895
Task A: Train loss: 0.3177923306822777, Val metrics: {'auroc': 0.6136910904712762, 'accuracy': 0.6, 'precision': 0.6512820512820513, 'recall': 0.679144385026738, 'f1score': 0.6649214659685864, 'loss': 1.0492538213729858}
Task B: Train loss: 0.2278740756213665, Val metrics: {'mae': 0.41462141720746887, 'r2': -0.4832272814058509, 'rmse': 0.5817730815311216, 'loss': 0.41462141275405884}


20it [00:05,  3.72it/s]


******************************<Epoch: 20>******************************
, Train loss: 0.02811640202999115, Val metrics: 1.4493356943130493
Task A: Train loss: 0.29670246243476867, Val metrics: {'auroc': 0.6069898453799383, 'accuracy': 0.6041666666666666, 'precision': 0.6596119929453262, 'recall': 0.6666666666666666, 'f1score': 0.6631205673758865, 'loss': 1.0379319190979004}
Task B: Train loss: 0.22655694410204888, Val metrics: {'mae': 0.4114037894582114, 'r2': -0.4728432313728941, 'rmse': 0.5797330127046392, 'loss': 0.4114038050174713}


In [28]:
best_epoch

7

In [29]:
# test the task a 
task_a_model.load_state_dict(state_dict_a)
test_logits = test(taska_loader_dict["test"], task_a_model, task_a)
test_logits = torch.sigmoid(test_logits).numpy()

test_pred = (test_logits > 0.5).astype(int)
test_pred_hat = task_a.get_table("test", mask_input_cols=False).df[task_a.target_col].to_numpy()
test_metrics = {
    "auroc": roc_auc_score(test_pred_hat, test_logits),
    "accuracy": accuracy_score(test_pred_hat, test_pred),
    "precision": precision_score(test_pred_hat, test_pred),
    "recall": recall_score(test_pred_hat, test_pred),
    "f1score": f1_score(test_pred_hat, test_pred),
}
test_metrics

{'auroc': 0.640629351155667,
 'accuracy': 0.6181818181818182,
 'precision': 0.6532846715328468,
 'recall': 0.7412008281573499,
 'f1score': 0.6944713870029098}

In [30]:
# test the task b
task_b_model.load_state_dict(state_dict_b)
logits = test(taskb_loader_dict["test"], task_b_model, task_b)
logits = logits.numpy()
pred_hat = task_b.get_table("test", mask_input_cols=False).df[task_b.target_col].to_numpy()
test_metrics = {
        "mae": mean_absolute_error(pred_hat, logits),
        "r2": r2_score(pred_hat, logits),
        "rmse": root_mean_squared_error(pred_hat, logits),
}
test_metrics

{'mae': 0.3995978656478217,
 'r2': -0.41550429739073613,
 'rmse': 0.5725317640168031}

: 